# 1. Initializations

## 1.1 General imports

In [ ]:
### general
import re
import string 

### data management
import pandas as pd
import numpy as np

### machine learning (scikit-learn)
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline 
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

### graphical
import matplotlib.pyplot as plt
# for jupyter notebook management
%matplotlib inline
import seaborn as sns


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 General classification functions

In [ ]:
# None

# 2. Loading and Data Quality

## 2.1 Loading of data sets and general exploration

In [ ]:
df_tit_raw = dfc.load_dataset_from_config('titanic_data', sep=',')

if df_tit_raw is not None and isinstance(df_tit_raw, pd.DataFrame):
    dfc.log_general_info(df_tit_raw)
    nb_first, nb_total = dfc.detect_and_log_duplicates_and_missing(df_tit_raw)
    if nb_first != nb_total:
        print(dfc.duplicates_index_map(df_tit_raw))
    df_tit = df_tit_raw.copy()
    display(df_tit.head())

In [ ]:
df_tit_desc = df_tit.select_dtypes(include=np.number).describe()
display(df_tit_desc)
df_tit_cr = df_tit.select_dtypes(include=np.number).corr()
display(df_tit_cr)

## 2.2 features and target split

In [ ]:
# Original backup
df_tit_orig = df_tit.copy()

In [ ]:
# Restore (optional)
df_tit = df_tit_orig.copy()

In [ ]:
y = df_tit['Survived']
X = df_tit.drop('Survived', axis=1)

# 3. Pipeline